In [1]:
import numpy as np

BOARD_ROWS = 3 ## 격자 행 3개
BOARD_COLS = 4 ## 격자 열 4개
WIN_STATE = (0, 3) ## 승리 지역은 (0,3)에 존재
LOSE_STATE = (1, 3) ## 패배 지역은 (1,3)에 존재
BLOCKED_STATE = (1, 1) ## (1,1)은 갈 수 없는 지역
START = (2, 0) ## 에이전트의 시작 지점은 (2,0)
DETERMINISTIC = False ## 설정한 격자는 비결정적인 환경으로 설정
class State: ## 상태 클래스 설정
    def __init__(self, state = START): ## 에피소드 초기화
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC

    def giveReward(self):
        if self.state == WIN_STATE: ## 게임 승리 시 1의 점수 획득
            return 1
        elif self.state == LOSE_STATE: ## 게임 패배 시 -1의 점수 획득
            return -1
        else: ## 그 외 점수 없음
            return 0

    def isEndFunc(self): ## 게임 종료인지 아닌지 판단하는 함수
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE): ## 게임 승리 혹은 패배 시 종료이므로
            self.isEnd = True ## 끝난 상태로 변경

    def _chooseActionProb(self, action): ## 액션 결정 확률
        if action == "U":
            return np.random.choice(["U", "L", "R"], p = [0.8, 0.1, 0.1]) ## 위로 가는 액션을 취했을 때, 의도대로 위로 갈 확률이 8할, 그렇지 않을 경우 각 1할씩. 이하의 경우 동일
        if action == "D":
            return np.random.choice(["D", "L", "R"], p = [0.8, 0.1, 0.1])
        if action == "L":
            return np.random.choice(["L", "U", "D"], p = [0.8, 0.1, 0.1])
        if action == "R":
            return np.random.choice(["R", "U", "D"], p = [0.8, 0.1, 0.1])

    def nxtPosition(self, action): ## 선택한 액션에 따른 상태 전이 계산
        if self.determine:
            if action == "U":
                nxtState = (self.state[0] - 1, self.state[1])
            elif action == "D":
                nxtState = (self.state[0] + 1, self.state[1])
            elif action == "L":
                nxtState = (self.state[0], self.state[1] - 1)
            else:
                nxtState = (self.state[0], self.state[1] + 1)
            self.determine = False
        else:
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)

        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != BLOCKED_STATE: ## 다음 지역이 이동 불가 지역이 아니라면
                    return nxtState ## 전이할 상태 반환
        return self.state ## 비정상적인 이동이 예상되면 상태 유지시킴

class Agent: ## 에이전트 클래스 설정

    def __init__(self): ## 초기화
        self.states = [] ## 상태 배열 초기화
        self.actions = ["U", "D", "L", "R"] ## 행동은 위, 아래, 왼, 오 이동만 가능
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2 ## 학습률 0.2
        self.decay_gamma = 0.9

        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0

    def chooseAction(self):
        max_nxt_reward = 0
        action = ""

        for a in self.actions:
            current_position = self.State.state
            nxt_reward = self.Q_values[current_position][a]
            if nxt_reward >= max_nxt_reward: ## 탐욕적 선택, 가장 큰 Q 값일 때만
                action = a
                max_nxt_reward = nxt_reward ## 최대 Q 값 업데이트
        return action

    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        return State(state = position)

    def reset(self): ## 격자, 게임 초기화
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd

    def play(self, episodes = 10):  ## 게임 시작, 에피소드는 10회
        i = 0 ## 에피소드 시도 횟수 초기화
        while i < episodes: ## 에피소드 횟수만큼 반복
            if self.State.isEnd:
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr * (self.decay_gamma * reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                self.states.append([(self.State.state), action])
                self.State = self.takeAction(action)
                self.State.isEndFunc()
                self.isEnd = self.State.isEnd

ag = Agent()

ag.play(1000) ## 게임 1000번 실행
print("latest Q-values ... \n") ## 상태에 따른 액션을 취했을 각 케이스 별로 학습된 최종 Q 값 출력
print(ag.Q_values)

latest Q-values ... 

{(0, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.627}, (0, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0.742}, (0, 2): {'U': 0, 'D': 0, 'L': 0, 'R': 0.86}, (0, 3): {'U': 1, 'D': 1, 'L': 1, 'R': 1}, (1, 0): {'U': 0, 'D': 0, 'L': 0, 'R': 0.086}, (1, 1): {'U': 0, 'D': 0, 'L': 0, 'R': 0}, (1, 2): {'U': 0, 'D': 0, 'L': 0.377, 'R': -0.18}, (1, 3): {'U': -1, 'D': -1, 'L': -1, 'R': -1}, (2, 0): {'U': 0, 'D': 0, 'L': 0.064, 'R': -0.001}, (2, 1): {'U': 0, 'D': 0, 'L': 0.11, 'R': -0.005}, (2, 2): {'U': 0, 'D': 0, 'L': 0.107, 'R': -0.028}, (2, 3): {'U': 0, 'D': 0, 'L': 0, 'R': -0.157}}
